No Aug Experiments - Mention increase in batch size to 500, as one will inflate batch size

In [3]:
from mlp.layers import MLP, Linear, Sigmoid, Softmax #import required layer types
from mlp.optimisers import SGDOptimiser #import the optimiser

from mlp.costs import CECost #import the cost we want to use for optimisation
from mlp.schedulers import LearningRateExponential, LearningRateFixed, LearningRateList, LearningRateNewBob

import numpy
import logging
import shelve
from mlp.dataset import MNISTDataProvider

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.info('Initialising data providers...')

#Set no aug
train_dp = MNISTDataProvider(dset='train', batch_size=50, max_num_batches=500, randomize=True, augmentation = 0)
valid_dp = MNISTDataProvider(dset='valid', batch_size=10000, max_num_batches=-10, randomize=False)
test_dp = MNISTDataProvider(dset='eval', batch_size=10000, max_num_batches=-10, randomize=False)

rng = numpy.random.RandomState([2015,10,10])

#some hyper-parameters
nhid = 800
max_epochs = 30
cost = CECost()
learning_rate = 0.5;
learningList = []
decrement = (learning_rate/max_epochs)

#Regulariser weights
l1_weight = 0.00
l2_weight = 0.000
dp_scheduler = None

#Build list once so we don't have to rebuild every time.
for i in xrange(0,max_epochs):
    #In this order so start learning rate is added
    learningList.append(learning_rate)
    learning_rate -= decrement



#Open file to save to
shelve_r = shelve.open("augExperiments")

stats = []
rate = 2

#For each number of layers, new model add layers.
for layer in xrange(0,2):
    #Set here in case we alter it in a layer experiment
    learning_rate = 0.5


    train_dp.reset()
    valid_dp.reset()
    test_dp.reset()

    logger.info("Starting ")

    #define the model
    model = MLP(cost=cost)

    if layer == 0:
        odim = 800
        model.add_layer(Sigmoid(idim=784, odim=odim, irange=0.2, rng=rng))
    if layer == 1:
        odim = 300
        model.add_layer(Sigmoid(idim=784, odim=odim, irange=0.2, rng=rng))
        model.add_layer(Sigmoid(idim=odim, odim=odim, irange=0.2, rng=rng))
        model.add_layer(Sigmoid(idim=odim, odim=odim, irange=0.2, rng=rng))
        model.add_layer(Sigmoid(idim=odim, odim=odim, irange=0.2, rng=rng))
        
    #Add output layer
    model.add_layer(Softmax(idim=odim, odim=10, rng=rng))

    #Set rate scheduler here
    if rate == 1:
        lr_scheduler = LearningRateExponential(start_rate=learning_rate, max_epochs=max_epochs, training_size=100)
    elif rate == 2:
        lr_scheduler = LearningRateFixed(learning_rate=learning_rate, max_epochs=max_epochs)
    elif rate == 3:
        # define the optimiser, here stochasitc gradient descent
        # with fixed learning rate and max_epochs
        lr_scheduler = LearningRateNewBob(start_rate=learning_rate, max_epochs=max_epochs,\
                                          min_derror_stop=.05, scale_by=0.05, zero_rate=learning_rate, patience = 10)

    optimiser = SGDOptimiser(lr_scheduler=lr_scheduler, 
                             dp_scheduler=dp_scheduler,
                             l1_weight=l1_weight, 
                             l2_weight=l2_weight)
    
    logger.info('Training started...')
    tr_stats, valid_stats = optimiser.train(model, train_dp, valid_dp)

    logger.info('Testing the model on test set:')
    tst_cost, tst_accuracy = optimiser.validate(model, test_dp)
    logger.info('MNIST test set accuracy is %.2f %%, cost (%s) is %.3f'%(tst_accuracy*100., cost.get_name(), tst_cost))

    #Append stats for all test
    stats.append((tr_stats, valid_stats, (tst_cost, tst_accuracy)))

    #Should save rate to specific dictionairy in pickle
    shelve_r['noAug'+str(layer)] = (tr_stats, valid_stats, (tst_cost, tst_accuracy))

logger.info('Saving Data')
shelve_r.close()   

INFO:root:Initialising data providers...
INFO:root:Starting 
INFO:root:Training started...
INFO:mlp.optimisers:Epoch 0: Training cost (ce) for initial model is 2.570. Accuracy is 9.32%
INFO:mlp.optimisers:Epoch 0: Validation cost (ce) for initial model is 2.554. Accuracy is 9.84%
INFO:mlp.optimisers:Epoch 1: Training cost (ce) is 0.805. Accuracy is 85.26%
INFO:mlp.optimisers:Epoch 1: Validation cost (ce) is 0.337. Accuracy is 89.83%
INFO:mlp.optimisers:Epoch 1: Took 39 seconds. Training speed 752 pps. Validation speed 1643 pps.
INFO:mlp.optimisers:Epoch 2: Training cost (ce) is 0.277. Accuracy is 91.73%
INFO:mlp.optimisers:Epoch 2: Validation cost (ce) is 0.239. Accuracy is 92.90%
INFO:mlp.optimisers:Epoch 2: Took 40 seconds. Training speed 748 pps. Validation speed 1601 pps.
INFO:mlp.optimisers:Epoch 3: Training cost (ce) is 0.222. Accuracy is 93.52%
INFO:mlp.optimisers:Epoch 3: Validation cost (ce) is 0.202. Accuracy is 94.34%
INFO:mlp.optimisers:Epoch 3: Took 39 seconds. Training sp

In [4]:
shelve_r = shelve.open("augExperiments")

print shelve_r['noAug0']

([(2.5703508233660521, 0.093239999999999837), (0.80523317568429009, 0.85255999999999876), (0.27677549487643593, 0.91727999999999832), (0.22214730183563638, 0.93519999999999825), (0.18305137563708659, 0.94583999999999857), (0.15569982539570845, 0.95375999999999872), (0.13300080456875954, 0.9615199999999996), (0.11569182332048453, 0.96659999999999957), (0.10137976111233382, 0.9708), (0.088637899782371021, 0.97419999999999984), (0.079508461866036467, 0.97772000000000003), (0.070032142996076019, 0.98096000000000105), (0.062405188919370608, 0.9832000000000013), (0.055755217898011723, 0.98532000000000119), (0.04967158365777917, 0.98744000000000121), (0.04458790914267511, 0.98912000000000144), (0.040571948966103173, 0.99020000000000119), (0.036383325059962189, 0.99172000000000138), (0.032378143263541291, 0.99300000000000133), (0.029633980765690616, 0.9940800000000013), (0.026752833110905212, 0.99552000000000074), (0.024419206568088901, 0.9958000000000008), (0.022283936445299839, 0.99644000000

Guassian augmentation

In [1]:
from mlp.layers import MLP, Linear, Sigmoid, Softmax #import required layer types
from mlp.optimisers import SGDOptimiser #import the optimiser

from mlp.costs import CECost #import the cost we want to use for optimisation
from mlp.schedulers import LearningRateExponential, LearningRateFixed, LearningRateList, LearningRateNewBob

import numpy
import logging
import shelve
from mlp.dataset import MNISTDataProvider

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.info('Initialising data providers...')

#Set no aug
train_dp = MNISTDataProvider(dset='train', batch_size=25, max_num_batches=250, randomize=True, augmentation = 1)
valid_dp = MNISTDataProvider(dset='valid', batch_size=10000, max_num_batches=-10, randomize=False)
test_dp = MNISTDataProvider(dset='eval', batch_size=10000, max_num_batches=-10, randomize=False)

rng = numpy.random.RandomState([2015,10,10])

#some hyper-parameters
nhid = 800
max_epochs = 30
cost = CECost()
learning_rate = 0.5;
learningList = []
decrement = (learning_rate/max_epochs)

#Regulariser weights
l1_weight = 0.00
l2_weight = 0.000
dp_scheduler = None

#Build list once so we don't have to rebuild every time.
for i in xrange(0,max_epochs):
    #In this order so start learning rate is added
    learningList.append(learning_rate)
    learning_rate -= decrement



#Open file to save to
shelve_r = shelve.open("gauAugExperimentsExp", writeback = True)

stats = []
rate = 2

#For each number of layers, new model add layers.
for layer in xrange(0,2):
    #Set here in case we alter it in a layer experiment
    learning_rate = 0.5


    train_dp.reset()
    valid_dp.reset()
    test_dp.reset()

    logger.info("Starting ")

    #define the model
    model = MLP(cost=cost)

    if layer == 0:
        odim = 800
        model.add_layer(Sigmoid(idim=784, odim=odim, irange=0.2, rng=rng))
    if layer == 1:
        odim = 300
        model.add_layer(Sigmoid(idim=784, odim=odim, irange=0.2, rng=rng))
        model.add_layer(Sigmoid(idim=odim, odim=odim, irange=0.2, rng=rng))
        model.add_layer(Sigmoid(idim=odim, odim=odim, irange=0.2, rng=rng))
        model.add_layer(Sigmoid(idim=odim, odim=odim, irange=0.2, rng=rng))
        
    #Add output layer
    model.add_layer(Softmax(idim=odim, odim=10, rng=rng))

    #Set rate scheduler here
    if rate == 1:
        lr_scheduler = LearningRateExponential(start_rate=learning_rate, max_epochs=max_epochs, training_size=100)
    elif rate == 2:
        lr_scheduler = LearningRateFixed(learning_rate=learning_rate, max_epochs=max_epochs)
    elif rate == 3:
        # define the optimiser, here stochasitc gradient descent
        # with fixed learning rate and max_epochs
        lr_scheduler = LearningRateNewBob(start_rate=learning_rate, max_epochs=max_epochs,\
                                          min_derror_stop=.05, scale_by=0.05, zero_rate=learning_rate, patience = 10)

    optimiser = SGDOptimiser(lr_scheduler=lr_scheduler, 
                             dp_scheduler=dp_scheduler,
                             l1_weight=l1_weight, 
                             l2_weight=l2_weight)
    
    logger.info('Training started...')
    tr_stats, valid_stats = optimiser.train(model, train_dp, valid_dp)

    logger.info('Testing the model on test set:')
    tst_cost, tst_accuracy = optimiser.validate(model, test_dp)
    logger.info('MNIST test set accuracy is %.2f %%, cost (%s) is %.3f'%(tst_accuracy*100., cost.get_name(), tst_cost))

    #Append stats for all test
    stats.append((tr_stats, valid_stats, (tst_cost, tst_accuracy)))

    #Should save rate to specific dictionairy in pickle
    shelve_r['gauAug'+str(layer)] = (tr_stats, valid_stats, (tst_cost, tst_accuracy))

logger.info('Saving Data')
shelve_r.close()   

INFO:root:Initialising data providers...
INFO:root:Starting 
INFO:root:Training started...
INFO:mlp.optimisers:Epoch 0: Training cost (ce) for initial model is 2.579. Accuracy is 9.04%
INFO:mlp.optimisers:Epoch 0: Validation cost (ce) for initial model is 2.554. Accuracy is 9.84%
INFO:mlp.optimisers:Epoch 1: Training cost (ce) is 1.301. Accuracy is 78.70%
INFO:mlp.optimisers:Epoch 1: Validation cost (ce) is 0.435. Accuracy is 86.18%
INFO:mlp.optimisers:Epoch 1: Took 17 seconds. Training speed 597 pps. Validation speed 1609 pps.
INFO:mlp.optimisers:Epoch 2: Training cost (ce) is 0.340. Accuracy is 89.79%
INFO:mlp.optimisers:Epoch 2: Validation cost (ce) is 0.328. Accuracy is 90.27%
INFO:mlp.optimisers:Epoch 2: Took 17 seconds. Training speed 583 pps. Validation speed 1536 pps.
INFO:mlp.optimisers:Epoch 3: Training cost (ce) is 0.273. Accuracy is 91.78%
INFO:mlp.optimisers:Epoch 3: Validation cost (ce) is 0.304. Accuracy is 91.06%
INFO:mlp.optimisers:Epoch 3: Took 17 seconds. Training sp

In [8]:
shelve_r = shelve.open("gauAugExperimentsExp")

print shelve_r['gauAug1']

([(2.3790590745646081, 0.097119999999999762), (2.3748161530123317, 0.11151999999999983), (1.6409201908863589, 0.38703999999999994), (0.7558383033078464, 0.74127999999999949), (0.47846208888599773, 0.84608000000000005), (0.39780283055768723, 0.8743999999999994), (0.33509262225273961, 0.89231999999999878), (0.29037617268422339, 0.90559999999999918), (0.2472264746783514, 0.92271999999999921), (0.21449407290921876, 0.93199999999999883), (0.19019722626937655, 0.93999999999999873), (0.16791916829606451, 0.94735999999999954), (0.14123214584240851, 0.95871999999999968), (0.12549076818379581, 0.96207999999999982), (0.10716737740252844, 0.96623999999999988), (0.099828401529748478, 0.97023999999999977), (0.083076926009576238, 0.97376000000000018), (0.067910550623422827, 0.97823999999999967), (0.060152025862470426, 0.98063999999999996), (0.052654247122175514, 0.98368000000000022), (0.041009519504898853, 0.98751999999999984), (0.037014065078455928, 0.98975999999999997), (0.028081005832056458, 0.992

Rotation Augmentation

In [1]:
from mlp.layers import MLP, Linear, Sigmoid, Softmax #import required layer types
from mlp.optimisers import SGDOptimiser #import the optimiser

from mlp.costs import CECost #import the cost we want to use for optimisation
from mlp.schedulers import LearningRateExponential, LearningRateFixed, LearningRateList, LearningRateNewBob

import numpy
import logging
import shelve
from mlp.dataset import MNISTDataProvider

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.info('Initialising data providers...')

#Set no aug
train_dp = MNISTDataProvider(dset='train', batch_size=25, max_num_batches=250, randomize=True, augmentation = 2)
valid_dp = MNISTDataProvider(dset='valid', batch_size=10000, max_num_batches=-10, randomize=False)
test_dp = MNISTDataProvider(dset='eval', batch_size=10000, max_num_batches=-10, randomize=False)

rng = numpy.random.RandomState([2015,10,10])

#some hyper-parameters
nhid = 800
max_epochs = 30
cost = CECost()
learning_rate = 0.5;
learningList = []
decrement = (learning_rate/max_epochs)

#Regulariser weights
l1_weight = 0.00
l2_weight = 0.000
dp_scheduler = None

#Build list once so we don't have to rebuild every time.
for i in xrange(0,max_epochs):
    #In this order so start learning rate is added
    learningList.append(learning_rate)
    learning_rate -= decrement



#Open file to save to
shelve_r = shelve.open("augExperiments")

stats = []
rate = 2

#For each number of layers, new model add layers.
for layer in xrange(0,2):
    #Set here in case we alter it in a layer experiment
    learning_rate = 0.5


    train_dp.reset()
    valid_dp.reset()
    test_dp.reset()

    logger.info("Starting ")

    #define the model
    model = MLP(cost=cost)

    if layer == 0:
        odim = 800
        model.add_layer(Sigmoid(idim=784, odim=odim, irange=0.2, rng=rng))
    if layer == 1:
        odim = 300
        model.add_layer(Sigmoid(idim=784, odim=odim, irange=0.2, rng=rng))
        model.add_layer(Sigmoid(idim=odim, odim=odim, irange=0.2, rng=rng))
        model.add_layer(Sigmoid(idim=odim, odim=odim, irange=0.2, rng=rng))
        model.add_layer(Sigmoid(idim=odim, odim=odim, irange=0.2, rng=rng))
        
    #Add output layer
    model.add_layer(Softmax(idim=odim, odim=10, rng=rng))

    #Set rate scheduler here
    if rate == 1:
        lr_scheduler = LearningRateExponential(start_rate=learning_rate, max_epochs=max_epochs, training_size=100)
    elif rate == 2:
        lr_scheduler = LearningRateFixed(learning_rate=learning_rate, max_epochs=max_epochs)
    elif rate == 3:
        # define the optimiser, here stochasitc gradient descent
        # with fixed learning rate and max_epochs
        lr_scheduler = LearningRateNewBob(start_rate=learning_rate, max_epochs=max_epochs,\
                                          min_derror_stop=.05, scale_by=0.05, zero_rate=learning_rate, patience = 10)

    optimiser = SGDOptimiser(lr_scheduler=lr_scheduler, 
                             dp_scheduler=dp_scheduler,
                             l1_weight=l1_weight, 
                             l2_weight=l2_weight)
    
    logger.info('Training started...')
    tr_stats, valid_stats = optimiser.train(model, train_dp, valid_dp)

    logger.info('Testing the model on test set:')
    tst_cost, tst_accuracy = optimiser.validate(model, test_dp)
    logger.info('MNIST test set accuracy is %.2f %%, cost (%s) is %.3f'%(tst_accuracy*100., cost.get_name(), tst_cost))

    #Append stats for all test
    stats.append((tr_stats, valid_stats, (tst_cost, tst_accuracy)))

    #Should save rate to specific dictionairy in pickle
    shelve_r['rotAug'+str(layer)] = (tr_stats, valid_stats, (tst_cost, tst_accuracy))

logger.info('Saving Data')
shelve_r.close()   

INFO:root:Initialising data providers...
INFO:root:Starting 
INFO:root:Training started...
INFO:mlp.optimisers:Epoch 0: Training cost (ce) for initial model is 2.556. Accuracy is 9.29%
INFO:mlp.optimisers:Epoch 0: Validation cost (ce) for initial model is 2.554. Accuracy is 9.84%
INFO:mlp.optimisers:Epoch 1: Training cost (ce) is 1.546. Accuracy is 70.56%
INFO:mlp.optimisers:Epoch 1: Validation cost (ce) is 0.531. Accuracy is 83.27%
INFO:mlp.optimisers:Epoch 1: Took 17 seconds. Training speed 528 pps. Validation speed 1839 pps.
INFO:mlp.optimisers:Epoch 2: Training cost (ce) is 0.522. Accuracy is 83.85%
INFO:mlp.optimisers:Epoch 2: Validation cost (ce) is 0.385. Accuracy is 88.80%
INFO:mlp.optimisers:Epoch 2: Took 18 seconds. Training speed 507 pps. Validation speed 1859 pps.
INFO:mlp.optimisers:Epoch 3: Training cost (ce) is 0.426. Accuracy is 87.08%
INFO:mlp.optimisers:Epoch 3: Validation cost (ce) is 0.328. Accuracy is 90.77%
INFO:mlp.optimisers:Epoch 3: Took 18 seconds. Training sp

In [9]:
shelve_r = shelve.open("augExperiments")

print shelve_r['rotAug0']

([(2.5562683854356489, 0.092879999999999921), (1.5462982589063765, 0.70559999999999989), (0.52189560304094995, 0.83848000000000067), (0.42601826261832176, 0.87079999999999957), (0.35337329232518222, 0.89584000000000008), (0.29377871768666169, 0.91415999999999931), (0.25147616865457145, 0.92511999999999917), (0.22159527630811471, 0.93327999999999944), (0.19857560650501502, 0.94423999999999864), (0.17065748879635703, 0.94991999999999932), (0.15739096503117883, 0.95615999999999868), (0.14088532762064696, 0.96007999999999927), (0.12800619988959416, 0.96423999999999932), (0.12028947523356515, 0.96695999999999938), (0.11102779961198238, 0.97127999999999903), (0.098303455248003366, 0.97311999999999876), (0.090107473644583214, 0.97663999999999851), (0.085240087924978794, 0.9765599999999991), (0.081285213966619668, 0.97903999999999891), (0.073936646915694307, 0.98167999999999878), (0.068661238422579068, 0.98223999999999922), (0.065987430741484179, 0.98383999999999849), (0.062576911035505542, 0.

Dropout Augmentation

In [2]:
from mlp.layers import MLP, Linear, Sigmoid, Softmax #import required layer types
from mlp.optimisers import SGDOptimiser #import the optimiser

from mlp.costs import CECost #import the cost we want to use for optimisation
from mlp.schedulers import LearningRateExponential, LearningRateFixed, LearningRateList, LearningRateNewBob

import numpy
import logging
import shelve
from mlp.dataset import MNISTDataProvider

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.info('Initialising data providers...')

#Set no aug
train_dp = MNISTDataProvider(dset='train', batch_size=25, max_num_batches=250, randomize=True, augmentation = 3)
valid_dp = MNISTDataProvider(dset='valid', batch_size=10000, max_num_batches=-10, randomize=False)
test_dp = MNISTDataProvider(dset='eval', batch_size=10000, max_num_batches=-10, randomize=False)

rng = numpy.random.RandomState([2015,10,10])

#some hyper-parameters
nhid = 800
max_epochs = 30
cost = CECost()
learning_rate = 0.5;
learningList = []
decrement = (learning_rate/max_epochs)

#Regulariser weights
l1_weight = 0.00
l2_weight = 0.000
dp_scheduler = None

#Build list once so we don't have to rebuild every time.
for i in xrange(0,max_epochs):
    #In this order so start learning rate is added
    learningList.append(learning_rate)
    learning_rate -= decrement



#Open file to save to
shelve_r = shelve.open("dpAugExperimentsExp", writeback = True)

stats = []
rate = 2

#For each number of layers, new model add layers.
for layer in xrange(0,2):
    #Set here in case we alter it in a layer experiment
    learning_rate = 0.5


    train_dp.reset()
    valid_dp.reset()
    test_dp.reset()

    logger.info("Starting ")

    #define the model
    model = MLP(cost=cost)

    if layer == 0:
        odim = 800
        model.add_layer(Sigmoid(idim=784, odim=odim, irange=0.2, rng=rng))
    if layer == 1:
        odim = 300
        model.add_layer(Sigmoid(idim=784, odim=odim, irange=0.2, rng=rng))
        model.add_layer(Sigmoid(idim=odim, odim=odim, irange=0.2, rng=rng))
        model.add_layer(Sigmoid(idim=odim, odim=odim, irange=0.2, rng=rng))
        model.add_layer(Sigmoid(idim=odim, odim=odim, irange=0.2, rng=rng))
        
    #Add output layer
    model.add_layer(Softmax(idim=odim, odim=10, rng=rng))

    #Set rate scheduler here
    if rate == 1:
        lr_scheduler = LearningRateExponential(start_rate=learning_rate, max_epochs=max_epochs, training_size=100)
    elif rate == 2:
        lr_scheduler = LearningRateFixed(learning_rate=learning_rate, max_epochs=max_epochs)
    elif rate == 3:
        # define the optimiser, here stochasitc gradient descent
        # with fixed learning rate and max_epochs
        lr_scheduler = LearningRateNewBob(start_rate=learning_rate, max_epochs=max_epochs,\
                                          min_derror_stop=.05, scale_by=0.05, zero_rate=learning_rate, patience = 10)

    optimiser = SGDOptimiser(lr_scheduler=lr_scheduler, 
                             dp_scheduler=dp_scheduler,
                             l1_weight=l1_weight, 
                             l2_weight=l2_weight)
    
    logger.info('Training started...')
    tr_stats, valid_stats = optimiser.train(model, train_dp, valid_dp)

    logger.info('Testing the model on test set:')
    tst_cost, tst_accuracy = optimiser.validate(model, test_dp)
    logger.info('MNIST test set accuracy is %.2f %%, cost (%s) is %.3f'%(tst_accuracy*100., cost.get_name(), tst_cost))

    #Append stats for all test
    stats.append((tr_stats, valid_stats, (tst_cost, tst_accuracy)))

    #Should save rate to specific dictionairy in pickle
    shelve_r['dpAug'+str(layer)] = (tr_stats, valid_stats, (tst_cost, tst_accuracy))

logger.info('Saving Data')
shelve_r.close()   

INFO:root:Initialising data providers...
INFO:root:Starting 
INFO:root:Training started...
INFO:mlp.optimisers:Epoch 0: Training cost (ce) for initial model is 2.579. Accuracy is 9.04%
INFO:mlp.optimisers:Epoch 0: Validation cost (ce) for initial model is 2.554. Accuracy is 9.84%
INFO:mlp.optimisers:Epoch 1: Training cost (ce) is 1.342. Accuracy is 77.98%
INFO:mlp.optimisers:Epoch 1: Validation cost (ce) is 0.372. Accuracy is 88.81%
INFO:mlp.optimisers:Epoch 1: Took 19 seconds. Training speed 498 pps. Validation speed 1462 pps.
INFO:mlp.optimisers:Epoch 2: Training cost (ce) is 0.338. Accuracy is 89.60%
INFO:mlp.optimisers:Epoch 2: Validation cost (ce) is 0.318. Accuracy is 90.68%
INFO:mlp.optimisers:Epoch 2: Took 19 seconds. Training speed 538 pps. Validation speed 1446 pps.
INFO:mlp.optimisers:Epoch 3: Training cost (ce) is 0.276. Accuracy is 91.50%
INFO:mlp.optimisers:Epoch 3: Validation cost (ce) is 0.268. Accuracy is 92.07%
INFO:mlp.optimisers:Epoch 3: Took 19 seconds. Training sp

In [7]:
shelve_r = shelve.open("dpAugExperimentsExp")

print shelve_r['dpAug1']

([(2.3790590745646085, 0.097119999999999734), (2.3881805605977542, 0.11215999999999977), (2.0159429450817039, 0.24991999999999995), (0.9488378819845753, 0.65952000000000055), (0.56004495608192173, 0.81999999999999995), (0.43044596275823094, 0.86431999999999976), (0.3591867620434484, 0.88607999999999898), (0.3134910808457535, 0.89887999999999935), (0.27059952836536155, 0.91359999999999841), (0.23750517326524775, 0.92463999999999913), (0.19672232450051497, 0.93711999999999929), (0.17857337194467107, 0.94671999999999878), (0.14944613340522162, 0.95359999999999967), (0.13530647543698449, 0.95680000000000009), (0.11397150207736742, 0.96431999999999995), (0.10661750135426776, 0.96607999999999994), (0.085781599460168917, 0.97392000000000023), (0.085140905375646961, 0.97568000000000044), (0.069225186659928897, 0.97904000000000013), (0.056527742859424662, 0.98112000000000021), (0.042557410028629845, 0.98768000000000056), (0.036852946536831258, 0.99008000000000007), (0.028968091205064595, 0.9923

Image Shift Augmentation

In [3]:
from mlp.layers import MLP, Linear, Sigmoid, Softmax #import required layer types
from mlp.optimisers import SGDOptimiser #import the optimiser

from mlp.costs import CECost #import the cost we want to use for optimisation
from mlp.schedulers import LearningRateExponential, LearningRateFixed, LearningRateList, LearningRateNewBob

import numpy
import logging
import shelve
from mlp.dataset import MNISTDataProvider

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.info('Initialising data providers...')

#Set no aug
train_dp = MNISTDataProvider(dset='train', batch_size=25, max_num_batches=250, randomize=True, augmentation = 4)
valid_dp = MNISTDataProvider(dset='valid', batch_size=10000, max_num_batches=-10, randomize=False)
test_dp = MNISTDataProvider(dset='eval', batch_size=10000, max_num_batches=-10, randomize=False)

rng = numpy.random.RandomState([2015,10,10])

#some hyper-parameters
nhid = 800
max_epochs = 30
cost = CECost()
learning_rate = 0.5;
learningList = []
decrement = (learning_rate/max_epochs)

#Regulariser weights
l1_weight = 0.00
l2_weight = 0.000
dp_scheduler = None

#Build list once so we don't have to rebuild every time.
for i in xrange(0,max_epochs):
    #In this order so start learning rate is added
    learningList.append(learning_rate)
    learning_rate -= decrement



#Open file to save to
shelve_r = shelve.open("siAugExperimentsExp", writeback = True)

stats = []
rate = 2

#For each number of layers, new model add layers.
for layer in xrange(0,2):
    #Set here in case we alter it in a layer experiment
    learning_rate = 0.5


    train_dp.reset()
    valid_dp.reset()
    test_dp.reset()

    logger.info("Starting ")

    #define the model
    model = MLP(cost=cost)

    if layer == 0:
        odim = 800
        model.add_layer(Sigmoid(idim=784, odim=odim, irange=0.2, rng=rng))
    if layer == 1:
        odim = 300
        model.add_layer(Sigmoid(idim=784, odim=odim, irange=0.2, rng=rng))
        model.add_layer(Sigmoid(idim=odim, odim=odim, irange=0.2, rng=rng))
        model.add_layer(Sigmoid(idim=odim, odim=odim, irange=0.2, rng=rng))
        model.add_layer(Sigmoid(idim=odim, odim=odim, irange=0.2, rng=rng))
        
    #Add output layer
    model.add_layer(Softmax(idim=odim, odim=10, rng=rng))

    #Set rate scheduler here
    if rate == 1:
        lr_scheduler = LearningRateExponential(start_rate=learning_rate, max_epochs=max_epochs, training_size=100)
    elif rate == 2:
        lr_scheduler = LearningRateFixed(learning_rate=learning_rate, max_epochs=max_epochs)
    elif rate == 3:
        # define the optimiser, here stochasitc gradient descent
        # with fixed learning rate and max_epochs
        lr_scheduler = LearningRateNewBob(start_rate=learning_rate, max_epochs=max_epochs,\
                                          min_derror_stop=.05, scale_by=0.05, zero_rate=learning_rate, patience = 10)

    optimiser = SGDOptimiser(lr_scheduler=lr_scheduler, 
                             dp_scheduler=dp_scheduler,
                             l1_weight=l1_weight, 
                             l2_weight=l2_weight)
    
    logger.info('Training started...')
    tr_stats, valid_stats = optimiser.train(model, train_dp, valid_dp)

    logger.info('Testing the model on test set:')
    tst_cost, tst_accuracy = optimiser.validate(model, test_dp)
    logger.info('MNIST test set accuracy is %.2f %%, cost (%s) is %.3f'%(tst_accuracy*100., cost.get_name(), tst_cost))

    #Append stats for all test
    stats.append((tr_stats, valid_stats, (tst_cost, tst_accuracy)))

    #Should save rate to specific dictionairy in pickle
    shelve_r['siAug'+str(layer)] = (tr_stats, valid_stats, (tst_cost, tst_accuracy))

logger.info('Saving Data')
shelve_r.close()   

INFO:root:Initialising data providers...
INFO:root:Starting 
INFO:root:Training started...
INFO:mlp.optimisers:Epoch 0: Training cost (ce) for initial model is 2.583. Accuracy is 8.58%
INFO:mlp.optimisers:Epoch 0: Validation cost (ce) for initial model is 2.554. Accuracy is 9.84%
INFO:mlp.optimisers:Epoch 1: Training cost (ce) is 1.743. Accuracy is 64.68%
INFO:mlp.optimisers:Epoch 1: Validation cost (ce) is 0.566. Accuracy is 82.79%
INFO:mlp.optimisers:Epoch 1: Took 19 seconds. Training speed 509 pps. Validation speed 1476 pps.
INFO:mlp.optimisers:Epoch 2: Training cost (ce) is 0.629. Accuracy is 81.30%
INFO:mlp.optimisers:Epoch 2: Validation cost (ce) is 0.412. Accuracy is 87.82%
INFO:mlp.optimisers:Epoch 2: Took 18 seconds. Training speed 571 pps. Validation speed 1455 pps.
INFO:mlp.optimisers:Epoch 3: Training cost (ce) is 0.500. Accuracy is 85.26%
INFO:mlp.optimisers:Epoch 3: Validation cost (ce) is 0.329. Accuracy is 91.31%
INFO:mlp.optimisers:Epoch 3: Took 18 seconds. Training sp

In [5]:
shelve_r = shelve.open("siAugExperimentsExp")

print shelve_r['siAug0']

([(2.5829580850249796, 0.085839999999999875), (1.7430002116563719, 0.64679999999999982), (0.62902550636653609, 0.81296000000000068), (0.50014567509224994, 0.85263999999999995), (0.4035487772071924, 0.88264000000000031), (0.35274842068266349, 0.89863999999999999), (0.30100346418398144, 0.91335999999999928), (0.2598261973346489, 0.92463999999999946), (0.23432229657982687, 0.93383999999999912), (0.21334208495475118, 0.9391199999999994), (0.19656201028969406, 0.944159999999999), (0.1774417412380915, 0.94975999999999927), (0.16347796582529248, 0.9529599999999987), (0.14495680404014663, 0.95999999999999952), (0.14157464820182641, 0.96103999999999867), (0.13083536725362305, 0.96495999999999871), (0.12079940992067965, 0.96599999999999864), (0.12074252957259465, 0.96711999999999909), (0.11185988883895207, 0.96999999999999953), (0.10393038768356934, 0.97215999999999858), (0.10013463862822616, 0.97407999999999872), (0.095163652272800922, 0.97487999999999841), (0.084308039195903225, 0.977839999999

Random Augmentation

In [1]:
from mlp.layers import MLP, Linear, Sigmoid, Softmax #import required layer types
from mlp.optimisers import SGDOptimiser #import the optimiser

from mlp.costs import CECost #import the cost we want to use for optimisation
from mlp.schedulers import LearningRateExponential, LearningRateFixed, LearningRateList, LearningRateNewBob

import numpy
import logging
import shelve
from mlp.dataset import MNISTDataProvider

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.info('Initialising data providers...')

#Set no aug
train_dp = MNISTDataProvider(dset='train', batch_size=25, max_num_batches=250, randomize=True, augmentation = 5)
valid_dp = MNISTDataProvider(dset='valid', batch_size=10000, max_num_batches=-10, randomize=False)
test_dp = MNISTDataProvider(dset='eval', batch_size=10000, max_num_batches=-10, randomize=False)

rng = numpy.random.RandomState([2015,10,10])

#some hyper-parameters
nhid = 800
max_epochs = 30
cost = CECost()
learning_rate = 0.5;
learningList = []
decrement = (learning_rate/max_epochs)

#Regulariser weights
l1_weight = 0.00
l2_weight = 0.000
dp_scheduler = None

#Build list once so we don't have to rebuild every time.
for i in xrange(0,max_epochs):
    #In this order so start learning rate is added
    learningList.append(learning_rate)
    learning_rate -= decrement



#Open file to save to
shelve_r = shelve.open("augExperiments")

stats = []
rate = 2

#For each number of layers, new model add layers.
for layer in xrange(0,2):
    #Set here in case we alter it in a layer experiment
    learning_rate = 0.5


    train_dp.reset()
    valid_dp.reset()
    test_dp.reset()

    logger.info("Starting ")

    #define the model
    model = MLP(cost=cost)

    if layer == 0:
        odim = 800
        model.add_layer(Sigmoid(idim=784, odim=odim, irange=0.2, rng=rng))
    if layer == 1:
        odim = 300
        model.add_layer(Sigmoid(idim=784, odim=odim, irange=0.2, rng=rng))
        model.add_layer(Sigmoid(idim=odim, odim=odim, irange=0.2, rng=rng))
        model.add_layer(Sigmoid(idim=odim, odim=odim, irange=0.2, rng=rng))
        model.add_layer(Sigmoid(idim=odim, odim=odim, irange=0.2, rng=rng))
        
    #Add output layer
    model.add_layer(Softmax(idim=odim, odim=10, rng=rng))

    #Set rate scheduler here
    if rate == 1:
        lr_scheduler = LearningRateExponential(start_rate=learning_rate, max_epochs=max_epochs, training_size=100)
    elif rate == 2:
        lr_scheduler = LearningRateFixed(learning_rate=learning_rate, max_epochs=max_epochs)
    elif rate == 3:
        # define the optimiser, here stochasitc gradient descent
        # with fixed learning rate and max_epochs
        lr_scheduler = LearningRateNewBob(start_rate=learning_rate, max_epochs=max_epochs,\
                                          min_derror_stop=.05, scale_by=0.05, zero_rate=learning_rate, patience = 10)

    optimiser = SGDOptimiser(lr_scheduler=lr_scheduler, 
                             dp_scheduler=dp_scheduler,
                             l1_weight=l1_weight, 
                             l2_weight=l2_weight)
    
    logger.info('Training started...')
    tr_stats, valid_stats = optimiser.train(model, train_dp, valid_dp)

    logger.info('Testing the model on test set:')
    tst_cost, tst_accuracy = optimiser.validate(model, test_dp)
    logger.info('MNIST test set accuracy is %.2f %%, cost (%s) is %.3f'%(tst_accuracy*100., cost.get_name(), tst_cost))

    #Append stats for all test
    stats.append((tr_stats, valid_stats, (tst_cost, tst_accuracy)))

    #Should save rate to specific dictionairy in pickle
    shelve_r['ranAug'+str(layer)] = (tr_stats, valid_stats, (tst_cost, tst_accuracy))

logger.info('Saving Data')
shelve_r.close()   

INFO:root:Initialising data providers...
INFO:root:Starting 
INFO:root:Training started...
INFO:mlp.optimisers:Epoch 0: Training cost (ce) for initial model is 2.576. Accuracy is 8.94%
INFO:mlp.optimisers:Epoch 0: Validation cost (ce) for initial model is 2.554. Accuracy is 9.84%
INFO:mlp.optimisers:Epoch 1: Training cost (ce) is 1.561. Accuracy is 71.10%
INFO:mlp.optimisers:Epoch 1: Validation cost (ce) is 0.430. Accuracy is 87.57%
INFO:mlp.optimisers:Epoch 1: Took 15 seconds. Training speed 640 pps. Validation speed 1878 pps.
INFO:mlp.optimisers:Epoch 2: Training cost (ce) is 0.546. Accuracy is 84.19%
INFO:mlp.optimisers:Epoch 2: Validation cost (ce) is 0.366. Accuracy is 89.49%
INFO:mlp.optimisers:Epoch 2: Took 16 seconds. Training speed 629 pps. Validation speed 1762 pps.
INFO:mlp.optimisers:Epoch 3: Training cost (ce) is 0.445. Accuracy is 87.37%
INFO:mlp.optimisers:Epoch 3: Validation cost (ce) is 0.303. Accuracy is 90.87%
INFO:mlp.optimisers:Epoch 3: Took 16 seconds. Training sp

In [13]:
shelve_r = shelve.open("augExperiments")

print shelve_r['ranAug0']

([(2.5764682227846394, 0.089439999999999811), (1.5609751983696472, 0.71104000000000056), (0.54552691188002445, 0.84192000000000022), (0.44474780014951787, 0.87368000000000046), (0.38089699144193384, 0.89104000000000005), (0.33450930787410083, 0.90312000000000048), (0.28111618041248881, 0.92231999999999958), (0.2445538690417324, 0.93287999999999927), (0.20922008233164727, 0.94103999999999932), (0.18941796677622746, 0.9469599999999988), (0.16984181609687674, 0.95503999999999878), (0.14939115243557247, 0.95903999999999889), (0.13943360034734933, 0.96479999999999866), (0.12470625478090855, 0.96559999999999924), (0.11335153012250314, 0.97271999999999914), (0.10506183880711599, 0.97263999999999884), (0.099271112707728537, 0.97599999999999931), (0.092739087160186875, 0.97847999999999846), (0.092797675219459252, 0.97863999999999884), (0.079184470418435765, 0.98039999999999861), (0.075331608062100874, 0.98255999999999832), (0.072937061354576366, 0.98151999999999895), (0.07320546749959457, 0.981

All Augmentations

In [1]:
from mlp.layers import MLP, Linear, Sigmoid, Softmax #import required layer types
from mlp.optimisers import SGDOptimiser #import the optimiser

from mlp.costs import CECost #import the cost we want to use for optimisation
from mlp.schedulers import LearningRateExponential, LearningRateFixed, LearningRateList, LearningRateNewBob

import numpy
import logging
import shelve
from mlp.dataset import MNISTDataProvider

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.info('Initialising data providers...')

#Set no aug
train_dp = MNISTDataProvider(dset='train', batch_size=10, max_num_batches=100, randomize=True, augmentation = 6)
valid_dp = MNISTDataProvider(dset='valid', batch_size=10000, max_num_batches=-10, randomize=False)
test_dp = MNISTDataProvider(dset='eval', batch_size=10000, max_num_batches=-10, randomize=False)

rng = numpy.random.RandomState([2015,10,10])

#some hyper-parameters
nhid = 800
max_epochs = 30
cost = CECost()
learning_rate = 0.5;
learningList = []
decrement = (learning_rate/max_epochs)

#Regulariser weights
l1_weight = 0.00
l2_weight = 0.000
dp_scheduler = None

#Build list once so we don't have to rebuild every time.
for i in xrange(0,max_epochs):
    #In this order so start learning rate is added
    learningList.append(learning_rate)
    learning_rate -= decrement



#Open file to save to
shelve_r = shelve.open("augExperiments")

stats = []
rate = 2

#For each number of layers, new model add layers.
for layer in xrange(0,2):
    #Set here in case we alter it in a layer experiment
    learning_rate = 0.5


    train_dp.reset()
    valid_dp.reset()
    test_dp.reset()

    logger.info("Starting ")

    #define the model
    model = MLP(cost=cost)

    if layer == 0:
        odim = 800
        model.add_layer(Sigmoid(idim=784, odim=odim, irange=0.2, rng=rng))
    if layer == 1:
        odim = 300
        model.add_layer(Sigmoid(idim=784, odim=odim, irange=0.2, rng=rng))
        model.add_layer(Sigmoid(idim=odim, odim=odim, irange=0.2, rng=rng))
        model.add_layer(Sigmoid(idim=odim, odim=odim, irange=0.2, rng=rng))
        model.add_layer(Sigmoid(idim=odim, odim=odim, irange=0.2, rng=rng))
        
    #Add output layer
    model.add_layer(Softmax(idim=odim, odim=10, rng=rng))

    #Set rate scheduler here
    if rate == 1:
        lr_scheduler = LearningRateExponential(start_rate=learning_rate, max_epochs=max_epochs, training_size=100)
    elif rate == 2:
        lr_scheduler = LearningRateFixed(learning_rate=learning_rate, max_epochs=max_epochs)
    elif rate == 3:
        # define the optimiser, here stochasitc gradient descent
        # with fixed learning rate and max_epochs
        lr_scheduler = LearningRateNewBob(start_rate=learning_rate, max_epochs=max_epochs,\
                                          min_derror_stop=.05, scale_by=0.05, zero_rate=learning_rate, patience = 10)

    optimiser = SGDOptimiser(lr_scheduler=lr_scheduler, 
                             dp_scheduler=dp_scheduler,
                             l1_weight=l1_weight, 
                             l2_weight=l2_weight)
    
    logger.info('Training started...')
    tr_stats, valid_stats = optimiser.train(model, train_dp, valid_dp)

    logger.info('Testing the model on test set:')
    tst_cost, tst_accuracy = optimiser.validate(model, test_dp)
    logger.info('MNIST test set accuracy is %.2f %%, cost (%s) is %.3f'%(tst_accuracy*100., cost.get_name(), tst_cost))

    #Append stats for all test
    stats.append((tr_stats, valid_stats, (tst_cost, tst_accuracy)))

    #Should save rate to specific dictionairy in pickle
    shelve_r['allAug'+str(layer)] = (tr_stats, valid_stats, (tst_cost, tst_accuracy))

logger.info('Saving Data')
shelve_r.close()   

INFO:root:Initialising data providers...
INFO:root:Starting 
INFO:root:Training started...
INFO:mlp.optimisers:Epoch 0: Training cost (ce) for initial model is 2.612. Accuracy is 8.54%
INFO:mlp.optimisers:Epoch 0: Validation cost (ce) for initial model is 2.554. Accuracy is 9.84%
INFO:mlp.optimisers:Epoch 1: Training cost (ce) is 3.339. Accuracy is 50.46%
INFO:mlp.optimisers:Epoch 1: Validation cost (ce) is 0.716. Accuracy is 78.08%
INFO:mlp.optimisers:Epoch 1: Took 9 seconds. Training speed 249 pps. Validation speed 1873 pps.
INFO:mlp.optimisers:Epoch 2: Training cost (ce) is 0.927. Accuracy is 71.14%
INFO:mlp.optimisers:Epoch 2: Validation cost (ce) is 0.526. Accuracy is 83.48%
INFO:mlp.optimisers:Epoch 2: Took 10 seconds. Training speed 236 pps. Validation speed 1753 pps.
INFO:mlp.optimisers:Epoch 3: Training cost (ce) is 0.740. Accuracy is 77.06%
INFO:mlp.optimisers:Epoch 3: Validation cost (ce) is 0.489. Accuracy is 85.75%
INFO:mlp.optimisers:Epoch 3: Took 10 seconds. Training spe

In [14]:
shelve_r = shelve.open("augExperiments")

print shelve_r['allAug0']

([(2.6121181065285213, 0.085399999999999934), (3.3386188054325641, 0.50460000000000005), (0.92744783493968019, 0.71140000000000014), (0.73978559840762348, 0.77060000000000017), (0.65191161521539442, 0.80840000000000001), (0.5356441589359745, 0.84120000000000017), (0.48156223556080574, 0.85779999999999967), (0.42952588115839468, 0.88200000000000012), (0.3837260789334026, 0.88980000000000004), (0.33331038473017471, 0.9111999999999999), (0.31116090080893472, 0.91519999999999957), (0.27474795056682294, 0.92039999999999988), (0.25136101986779752, 0.9353999999999999), (0.21947064786448864, 0.94119999999999937), (0.19793114700923031, 0.94539999999999946), (0.17974359308773238, 0.95379999999999943), (0.18305838239454297, 0.95039999999999936), (0.16257642352225748, 0.95699999999999985), (0.16464408652818271, 0.95400000000000007), (0.1438111786511668, 0.9611999999999995), (0.14314446972757594, 0.96219999999999939), (0.12716693398533691, 0.96859999999999946), (0.13241150587049533, 0.9627999999999

### All with improved rates.

In [16]:
from mlp.layers import MLP, Linear, Sigmoid, Softmax #import required layer types
from mlp.optimisers import SGDOptimiser #import the optimiser

from mlp.costs import CECost #import the cost we want to use for optimisation
from mlp.schedulers import LearningRateExponential, LearningRateFixed, LearningRateList, LearningRateNewBob, DropoutAnnealed

import numpy
import logging
import shelve
from mlp.dataset import MNISTDataProvider

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.info('Initialising data providers...')

#Set no aug
train_dp = MNISTDataProvider(dset='train', batch_size=100, max_num_batches=1000, randomize=True, augmentation = 6)
valid_dp = MNISTDataProvider(dset='valid', batch_size=10000, max_num_batches=-10, randomize=False)
test_dp = MNISTDataProvider(dset='eval', batch_size=10000, max_num_batches=-10, randomize=False)

rng = numpy.random.RandomState([2015,10,10])

#some hyper-parameters
nhid = 800
max_epochs = 30
cost = CECost()
learning_rate = 0.5;
learningList = []
decrement = (learning_rate/max_epochs)

#Regulariser weights
l1_weight = 0.00
'''
    Picked l2, as l1 can eliminate some points, which we are doing anyway with dropout, whilst l2 will just shrink 
    once annealing (dropout) gets closer to 1.
'''
#l2_weight = 0.00005

#Build list once so we don't have to rebuild every time.
for i in xrange(0,max_epochs):
    #In this order so start learning rate is added
    learningList.append(learning_rate)
    learning_rate -= decrement



#Open file to save to
shelve_r = shelve.open("allAugExperimentsBet", writeback = True)

stats = []
rate = 2


'''
Since we will have a lot more data, make smaller than normal
'''
dp_scheduler = None


#For each number of layers, new model add layers.
for layer in xrange(0,2):
    #Set here in case we alter it in a layer experiment
    learning_rate = 0.5


    train_dp.reset()
    valid_dp.reset()
    test_dp.reset()

    logger.info("Starting ")

    #define the model
    model = MLP(cost=cost)

    if layer == 0:
        odim = 800
        model.add_layer(Sigmoid(idim=784, odim=odim, irange=0.2, rng=rng))
    if layer == 1:
        odim = 300
        model.add_layer(Sigmoid(idim=784, odim=odim, irange=0.2, rng=rng))
        model.add_layer(Sigmoid(idim=odim, odim=odim, irange=0.2, rng=rng))
        model.add_layer(Sigmoid(idim=odim, odim=odim, irange=0.2, rng=rng))
        model.add_layer(Sigmoid(idim=odim, odim=odim, irange=0.2, rng=rng))
        
    #Add output layer
    model.add_layer(Softmax(idim=odim, odim=10, rng=rng))

    #Set rate scheduler here
    if rate == 1:
        lr_scheduler = LearningRateExponential(start_rate=learning_rate, max_epochs=max_epochs, training_size=100)
    elif rate == 2:
        lr_scheduler = LearningRateFixed(learning_rate=learning_rate, max_epochs=max_epochs)
    elif rate == 3:
        # define the optimiser, here stochasitc gradient descent
        # with fixed learning rate and max_epochs
        lr_scheduler = LearningRateNewBob(start_rate=learning_rate, max_epochs=max_epochs,\
                                          min_derror_stop=.05, scale_by=0.05, zero_rate=learning_rate, patience = 10)

    optimiser = SGDOptimiser(lr_scheduler=lr_scheduler, 
                             dp_scheduler=dp_scheduler,
                             l1_weight=l1_weight, 
                             l2_weight=l2_weight)
    
    logger.info('Training started...')
    tr_stats, valid_stats = optimiser.train(model, train_dp, valid_dp)

    logger.info('Testing the model on test set:')
    tst_cost, tst_accuracy = optimiser.validate(model, test_dp)
    logger.info('MNIST test set accuracy is %.2f %%, cost (%s) is %.3f'%(tst_accuracy*100., cost.get_name(), tst_cost))

    #Append stats for all test
    stats.append((tr_stats, valid_stats, (tst_cost, tst_accuracy)))

    #Should save rate to specific dictionairy in pickle
    shelve_r['allAug'+str(layer)] = (tr_stats, valid_stats, (tst_cost, tst_accuracy))

logger.info('Saving Data')
shelve_r.close()   

INFO:root:Initialising data providers...
INFO:root:Starting 
INFO:root:Training started...
INFO:mlp.optimisers:Epoch 0: Training cost (ce) for initial model is 2.773. Accuracy is 9.29%
INFO:mlp.optimisers:Epoch 0: Validation cost (ce) for initial model is 2.763. Accuracy is 9.84%
INFO:mlp.optimisers:Epoch 1: Training cost (ce) is 1.203. Accuracy is 79.53%
INFO:mlp.optimisers:Epoch 1: Validation cost (ce) is 0.547. Accuracy is 90.61%
INFO:mlp.optimisers:Epoch 1: Took 121 seconds. Training speed 419 pps. Validation speed 4639 pps.
INFO:mlp.optimisers:Epoch 2: Training cost (ce) is 0.627. Accuracy is 88.18%
INFO:mlp.optimisers:Epoch 2: Validation cost (ce) is 0.459. Accuracy is 93.66%
INFO:mlp.optimisers:Epoch 2: Took 121 seconds. Training speed 420 pps. Validation speed 4502 pps.
INFO:mlp.optimisers:Epoch 3: Training cost (ce) is 0.538. Accuracy is 91.01%
INFO:mlp.optimisers:Epoch 3: Validation cost (ce) is 0.422. Accuracy is 94.68%
INFO:mlp.optimisers:Epoch 3: Took 117 seconds. Training